In [ ]:
# default_exp workflow
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Workflow

> Define static or dynamic workflow for automatically updating, training and deploying your ML model!



***input:*** Workflow definition parameters

***output:*** python or snakemake script for running the workflow

***description:***

In this notebook you define a workflow for automatically updating, training and deploying your ML project

## Import relevant modules

In [ ]:
import numpy as np
import pandas as pd 
import papermill as pm

In [ ]:
seed = 0

In [ ]:
np.random.seed(0)

## Custom magic function to write code cell contents to a file with global variables formatted:

This is needed so that we can create parameterizable non-python scripts from inside a notebook.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def write_format_globals(filename, cell):
    """
    This is a function to write contents of a notebook cell to a file.
    To use it, call '%write_format_globals filename' in the first row of
    the cell the contents of which you want to write into a file.
    The code written in this cell is not run in the notebook.
    This means that you can also write and define non-python scripts and
    execute them from a notebook.

    You can format in global variables by placing them inside curly 
    brackets '{variable name}'.

    Note, that your code should not include curly brackets.
    If curly brackets need to be written in the file, you can include 
    them through the variable insertion:

    ## CELL 1:
    bracket_open_string = '{'
    bracket_close_string = '}'

    ## CELL 2:
    %write_format_globals myscriptname
    {bracket_open_string}
        # I want this part inside curly brackets!
    {bracket_close_string}
    
    ## file 'myscriptname' after running the cells 1 and 2:
    {
        # I want this part inside curly brackets!
    }
    """
    with open(filename, "w") as f:
        f.write(cell.format(**globals()))

## Example: running static workflow from this notebook

(Snakemake-based dynamic workflow can not be run from inside a notebook) 

## Alternative 1: Static Executable Workflow with papermill

The following script will be written into a python file, that you can run to execute the workflow.

## Alternative 2: Dynamic Executable Workflow with Snakemake

The following script will be written into a Snakefile, that you can run to execute the workflow.